In [ ]:
# Install the libraries again
!pip install pycaret

In [ ]:
import pandas as pd

df = pd.read_csv("/content/ml_acidentes_mensal_full.csv", sep=',', low_memory=False)

In [ ]:
# 1. Setup do PyCaret
from pycaret.regression import *

setup(data=df, target='qtd_acidentes', session_id=42)

# 2. Treinar e comparar modelos
best_model = compare_models()

# 3. Fazer previsões no holdout/teste
predictions = predict_model(best_model)

# 4. Criar DataFrame com Real vs Previsto
resultado = predictions[['qtd_acidentes', 'prediction_label']]
resultado.columns = ['Real', 'Previsto']

# 5. Mostrar os primeiros resultados
print(resultado.head(20))

,Description,Value
0,Session id,42
1,Target,qtd_acidentes
2,Target type,Regression
3,Original data shape,"(83931, 15)"
4,Transformed data shape,"(83931, 15)"
5,Transformed train set shape,"(58751, 15)"
6,Transformed test set shape,"(25180, 15)"
7,Numeric features,12
8,Categorical features,2
9,Rows with missing values,0.3%


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0576,0.0413,0.2028,0.8311,0.0682,0.0388,6.1930
lightgbm,Light Gradient Boosting Machine,0.0639,0.0436,0.2082,0.8219,0.0666,0.0404,2.5280
xgboost,Extreme Gradient Boosting,0.0685,0.0439,0.2091,0.8204,0.0686,0.0447,0.5230
gbr,Gradient Boosting Regressor,0.0878,0.0533,0.2305,0.7816,0.0755,0.0586,4.2360
rf,Random Forest Regressor,0.0794,0.0568,0.2379,0.7675,0.0791,0.0527,11.4860
dt,Decision Tree Regressor,0.0752,0.1078,0.3275,0.5588,0.1097,0.0482,0.3330
lar,Least Angle Regression,0.2270,0.2051,0.4527,0.1591,0.1539,0.1637,0.1750
br,Bayesian Ridge,0.2270,0.2051,0.4527,0.1591,0.1539,0.1637,0.1860
ridge,Ridge Regression,0.2270,0.2051,0.4527,0.1591,0.1539,0.1637,0.1820
lr,Linear Regression,0.2270,0.2051,0.4527,0.1591,0.1539,0.1637,0.9070


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.0557,0.0383,0.1958,0.8460,0.0661,0.0378


       Real  Previsto
53456     1      1.00
65943     1      1.00
54906     1      1.00
64019     1      1.00
16170     1      1.00
59862     1      1.00
9026      3      2.80
50544     1      1.35
69440     1      1.04
69245     1      1.01
83674     1      1.01
21592     1      1.01
43493     1      1.00
49763     1      1.00
16617     1      1.01
53200     1      1.01
32978     1      1.03
35594     1      1.00
44360     1      1.03
74378     2      2.56


In [ ]:
import pandas as pd
from pycaret.regression import *

# 1. Filtra os dados
df_historica = df[df['ano'].isin([2020,2021,2022])].copy()
df_atual = df[df['ano'].isin([2023,2024,2025])].copy()

# 2. Configura o ambiente com os dados de 2024
setup(data=df_historica, target='qtd_acidentes', session_id=123)

# 3. Treina o melhor modelo
best_model = compare_models()

# 4. Faz a predição usando os dados de 2025
predictions = predict_model(best_model, data=df_atual)

# 5. Exibe os resultados
predictions[['qtd_acidentes', 'prediction_label']].head()

,Description,Value
0,Session id,123
1,Target,qtd_acidentes
2,Target type,Regression
3,Original data shape,"(173578, 15)"
4,Transformed data shape,"(173578, 15)"
5,Transformed train set shape,"(121504, 15)"
6,Transformed test set shape,"(52074, 15)"
7,Numeric features,12
8,Categorical features,2
9,Rows with missing values,0.3%


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0464,0.0283,0.1682,0.8401,0.0609,0.0328,21.6150
lightgbm,Light Gradient Boosting Machine,0.0503,0.0288,0.1696,0.8374,0.0588,0.0332,6.5620
xgboost,Extreme Gradient Boosting,0.0529,0.0292,0.1706,0.8355,0.0600,0.0355,1.3260
rf,Random Forest Regressor,0.0584,0.0357,0.1888,0.7988,0.0675,0.0406,37.5380
gbr,Gradient Boosting Regressor,0.0690,0.0374,0.1933,0.7890,0.0665,0.0466,13.0520
dt,Decision Tree Regressor,0.0562,0.0735,0.2711,0.5847,0.0962,0.0386,1.0550
lr,Linear Regression,0.1888,0.1536,0.3919,0.1332,0.1371,0.1375,1.4140
ridge,Ridge Regression,0.1888,0.1536,0.3919,0.1332,0.1371,0.1375,0.4450
br,Bayesian Ridge,0.1888,0.1536,0.3919,0.1332,0.1371,0.1375,0.4500
lar,Least Angle Regression,0.1890,0.1537,0.3919,0.1330,0.1371,0.1377,0.5610


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.2503,0.2653,0.5151,-0.0988,0.1761,0.1772


,qtd_acidentes,prediction_label
173578,2,1.03
173579,1,1.15
173580,1,1.08
173581,1,1.07
173582,1,1.12


In [ ]:
import numpy as np

# Garantir que 'prediction_label' seja numérico
predictions['prediction_label'] = pd.to_numeric(predictions['prediction_label'], errors='coerce')

predictions['br'] = predictions['prediction_label'].astype(str)

predictions['media_idade_envolvidos'] = np.floor(predictions['media_idade_envolvidos']).astype(int)


# Criar colunas com arredondamento
predictions['pred_arredondado_baixo'] = np.floor(predictions['prediction_label']).astype(int)

# Visualizar as novas colunas
predictions[['prediction_label', 'pred_arredondado_baixo']].head()


,prediction_label,pred_arredondado_baixo
173578,1.03,1
173579,1.15,1
173580,1.08,1
173581,1.07,1
173582,1.12,1


In [ ]:
import csv
from google.colab import files

# Nome do arquivo
filename = 'previsoes_modelo_full.csv'

# Abre o arquivo e escreve com o módulo csv
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Escreve o cabeçalho
    writer.writerow(predictions.columns.tolist())

    # Escreve os dados
    for row in predictions.itertuples(index=False):
        writer.writerow(row)

# Faz o download
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>